In [ ]:
import cv2
import pickle

# ---------- LOAD SLOT POSITIONS ----------
with open("ParkingSlotPoints.pkl", "rb") as f:
    points = pickle.load(f)

# ---------- VIDEO CAPTURE ----------
cap = cv2.VideoCapture("sample.mp4")

# ---------- SLOT SIZE ----------
slot_width = 100
slot_height = 44

# ---------- VIDEO WRITER ----------
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Or use 'XVID'
out = cv2.VideoWriter("result_video.mp4", fourcc, fps, (frame_width, frame_height))

def check_parking_space(img_process, img_draw):
    free_space_count = 0

    for point in points:
        x, y = point[0]
        crop = img_process[y:y+slot_height, x:x+slot_width]
        count = cv2.countNonZero(crop)

        if count < 900:
            color = (0, 255, 0)  # green
            thickness = 2
            free_space_count += 1
        else:
            color = (0, 0, 255)  # red
            thickness = 2

        cv2.rectangle(img_draw, (x, y), (x+slot_width, y+slot_height), color, thickness)

    # Show availability counter
    total = len(points)
    cv2.rectangle(img_draw, (10, 10), (220, 50), (0, 0, 0), -1)
    cv2.putText(img_draw, f"Free: {free_space_count}/{total}", (15, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    # Processing
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 1)
    img_thresh = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 25, 16)
    img_median = cv2.medianBlur(img_thresh, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_dilate = cv2.dilate(img_median, kernel, iterations=1)

    # Check slots
    check_parking_space(img_dilate, frame)

    # Write and show
    out.write(frame)
    cv2.imshow("Parking Detection", frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
